# Telegram

本笔记本展示如何使用Telegram聊天加载器。该类可以帮助将导出的Telegram对话映射到LangChain聊天消息。

该过程包括三个步骤：
1. 通过从Telegram应用程序复制聊天内容并将其粘贴到本地计算机上的文件中来导出聊天.txt文件
2. 创建`TelegramChatLoader`，并将文件路径指向JSON文件或包含JSON文件的目录
3. 调用`loader.load()`（或`loader.lazy_load()`）执行转换。可以选择使用`merge_chat_runs`来合并来自同一发送者的连续消息，和/或使用`map_ai_messages`将指定发送者的消息转换为"AIMessage"类。

## 1. 创建消息导出

目前（2023/08/23）此加载器最适合支持从[Telegram Desktop应用](https://desktop.telegram.org/)导出的聊天历史记录JSON格式文件。

**重要提示：**有一些Telegram的"精简版"（如"Telegram for MacOS"）缺少导出功能。请确保使用正确的应用程序导出文件。

导出步骤：
1. 下载并打开Telegram桌面版
2. 选择一个对话
3. 导航到对话设置（当前为右上角的三个点）
4. 点击"导出聊天记录"
5. 取消选择照片和其他媒体。选择"机器可读JSON"格式进行导出。

以下是示例：

In [1]:
%%writefile telegram_conversation.json
{
 "name": "Jiminy",
 "type": "personal_chat",
 "id": 5965280513,
 "messages": [
  {
   "id": 1,
   "type": "message",
   "date": "2023-08-23T13:11:23",
   "date_unixtime": "1692821483",
   "from": "Jiminy Cricket",
   "from_id": "user123450513",
   "text": "You better trust your conscience",
   "text_entities": [
    {
     "type": "plain",
     "text": "You better trust your conscience"
    }
   ]
  },
  {
   "id": 2,
   "type": "message",
   "date": "2023-08-23T13:13:20",
   "date_unixtime": "1692821600",
   "from": "Batman & Robin",
   "from_id": "user6565661032",
   "text": "What did you just say?",
   "text_entities": [
    {
     "type": "plain",
     "text": "What did you just say?"
    }
   ]
  }
 ]
}

Overwriting telegram_conversation.json


## 2. 创建聊天加载器

只需要提供文件路径。您还可以选择指定映射到AI消息的用户名，以及配置是否合并消息序列。

In [2]:
from langchain_community.chat_loaders.telegram import TelegramChatLoader

In [3]:
loader = TelegramChatLoader(
    path="./telegram_conversation.json",
)

## 3. 加载消息

`load()`（或`lazy_load`）方法返回"ChatSessions"列表，目前每个加载的对话仅包含一个消息列表。

In [4]:
from typing import List

from langchain_community.chat_loaders.utils import (
    map_ai_messages,
    merge_chat_runs,
)
from langchain_core.chat_sessions import ChatSession

raw_messages = loader.lazy_load()
# Merge consecutive messages from the same sender into a single message
merged_messages = merge_chat_runs(raw_messages)
# Convert messages from "Jiminy Cricket" to AI messages
messages: List[ChatSession] = list(
    map_ai_messages(merged_messages, sender="Jiminy Cricket")
)

### 后续步骤

然后，您可以根据需要使用这些消息，例如微调模型、少量示例选择或直接为下一条消息进行预测

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

for chunk in llm.stream(messages[0]["messages"]):
    print(chunk.content, end="", flush=True)

I said, "You better trust your conscience."